In [ ]:
!pip install sentencepiece
!pip install tf-models-nightly
!pip install tf-nightly

     |████████████████████████████████| 1.2MB 7.2MB/s 
     |████████████████████████████████| 1.2MB 7.4MB/s 
     |████████████████████████████████| 276kB 49.7MB/s 
     |████████████████████████████████| 399.9MB 39kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 174kB 56.8MB/s 
     |████████████████████████████████| 37.6MB 1.2MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 358kB 46.1MB/s 
     |████████████████████████████████| 12.2MB 52.2MB/s 
     |████████████████████████████████| 4.0MB 48.7MB/s 
     |████████████████████████████████| 471kB 56.1MB/s 
     |████████████████████████████████| 4.0MB 37.3MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=5850d8353db04b3bf56c2dc4206f72ccdf73496db11bd350bc90a2c06075dfff
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Creat

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0-dev20210110'

In [ ]:
import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file

from official.nlp import optimization

from official.nlp.data.squad_lib import (
    read_squad_examples, FeatureWriter,
    convert_examples_to_features, write_predictions)

In [ ]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

## Data processing

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/MyDrive/BERT/data/squad/train-v1.1.json",
    "/content/drive/MyDrive/BERT/data/squad/vocab.txt",
    "/content/drive/MyDrive/BERT/data/squad/train-v1.1.tf_record"
)

In [ ]:
with tf.io.gfile.GFile("/content/drive/MyDrive/BERT/data/squad/train_meta_data.gfile", "w") as writer:
  writer.write(json.dumps(input_meta_data, indent=4) + '\n')

In [ ]:
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/drive/MyDrive/BERT/data/squad/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=False
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Model building

## squad layer

In [ ]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)
    )

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)]
    return unstacked_logits[0], unstacked_logits[1]

## whole model

In [ ]:
class BertSquad(tf.keras.Model):

  def __init__(self, name="bert_squad"):
    super(BertSquad, self).__init__(name=name)
    self.bert_layer = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
        trainable=True
    )
    self.squad_layer = BertSquadLayer()

  def apply_bert(self, inputs):
    _, sequence_output = self.bert_layer(
        [inputs["input_word_ids"],
        inputs["input_mask"],
        inputs["input_type_ids"]]
    )
    return sequence_output

  def call(self, inputs):
    seq_output = self.apply_bert(inputs)
    start_logits, end_logits = self.squad_layer(seq_output)
    return start_logits, end_logits

In [ ]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    def apply_bert(self, inputs):
#        _ , sequence_output = self.bert_layer([inputs["input_ids"],
#                                               inputs["input_mask"],
#                                               inputs["segment_ids"]])
        
        # New names for the 3 different elements of the inputs, since an update
        # in tf_models_officials. Doesn't change anything for any other BERT
        # usage.
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

# Training

## creating the AI

In [ ]:
TRAIN_DATA_SIZE = 88641 # this is our training dataset
NB_BATCHES_TRAIN = 2000 # we will not use the full one as fine tuning bert is quite heavy
BATCH_SIZE = 4
NB_EPOCHS = 3 # 2 should be fine but we are using 3
INIT_LR = 5e-4 # we are going to use a different optimiser that is given by google specifically for BERT
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [ ]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [ ]:
bert_squad = BertSquad()

In [ ]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS
)

In [ ]:
def squad_loss_function(labels, model_outputs):
  start_positions = labels["start_positions"]
  end_positions = labels["end_positions"]
  start_logits, end_logits = model_outputs

  start_loss = tf.keras.backend.sparse_categorical_crossentropy(
      start_positions, start_logits, from_logits=True
  )
  end_loss = tf.keras.backend.sparse_categorical_crossentropy(
      end_positions, end_logits, from_logits=True
  )

  total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2
  return total_loss

train_loss = tf.keras.metrics.Mean(name='train_loss')

In [ ]:
next(iter(train_dataset_light))

({'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[  101,  2000,  3183, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0],
         [  101,  1996, 13546, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0]], dtype=int32)>,
  'unique_ids': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([1000000000, 1000000001, 1000000002, 1000000003], dtype=int32)>},
 {})

In [ ]:
bert_squad.compile(optimizer, squad_loss_function)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/BERT/ckpt_bert_squad"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
for epoch in range(NB_EPOCHS):
  print("start of epoch {}".format(epoch + 1))
  start = time.time()

  train_loss.reset_states()

  for (batch, (inputs, targets)) in enumerate(train_dataset_light):
    with tf.GradientTape() as tape:
      model_outputs = bert_squad(inputs)
      loss = squad_loss_function(targets, model_outputs)

    gradients = tape.gradient(loss, bert_squad.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))

    train_loss(loss)

    if batch % 50 == 0:
      print("Epoch {} Batch {} Loss {:.4f}".format(
          epoch+1, batch, train_loss.result()))
      
      if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                               ckpt_save_path))
  print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

start of epoch 1


ValueError: ignored